In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)


# Build Classic Transform

Use standard field to calculate photometric transform by classic method ("slope of slopes")

In [ ]:
import vso.util
import vso.data
import vso.phot
import numpy as np

OBJECTS = ['SA20', 'SA38', 'SA41']
SESSION_TAG='2024/20241003'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

layout = vso.util.WorkLayout(WORK_ROOT)
sd = vso.data.StarData(layout.charts_dir)


In [ ]:
from astropy.table import QTable, Column, vstack, join

bands = ('B', 'V')
def get_session_data(name, tag, bands):
    session = vso.util.Session(tag=tag, name=name)
    layout = vso.util.WorkLayout(WORK_ROOT)
    session_layout = layout.get_session(session)
    provider = vso.phot.BatchDataProvider(session_layout)
    return provider.batch_and_sequence_band_pair(bands)


In [ ]:
from vso.phot import ValErr, ValErrDtype
from scipy import stats as sst
from collections import namedtuple

PointClassicTransform = namedtuple('PointClassicTransform', ['Ta', 'Za', 'Tb', 'Zb'])

def fiilter_std(xfm, X, scale=1):
    limit = np.mean(xfm[X]['err']) + scale*np.std(xfm[X]['err'])
    return xfm[X]['err'] < limit

def fiilter_std_all(xfm, scale=1):
    return fiilter_std(xfm, 'Ta', scale) & fiilter_std(xfm, 'Za', scale) & fiilter_std(xfm, 'Tb', scale) & fiilter_std(xfm, 'Zb', scale)

def calc_batched_classic_transform(name, tag, bands):
    def batch_regression(batch, bands):
        ci = batch[bands[0]]['mag'] - batch[bands[1]]['mag']
        dA = batch[bands[0]]['mag'] - batch[f"instr {bands[0]}"]['mag']
        dB = batch[bands[1]]['mag'] - batch[f"instr {bands[1]}"]['mag']
        regA = sst.linregress(ci, dA)
        regB = sst.linregress(ci, dB)
        return PointClassicTransform(ValErr(regA.slope, regA.stderr),
                                ValErr(regA.intercept, regA.intercept_stderr),
                                ValErr(regB.slope, regB.stderr),
                                ValErr(regB.intercept, regB.intercept_stderr))
    session = vso.util.Session(tag=tag, name=name)
    layout = vso.util.WorkLayout(WORK_ROOT)
    session_layout = layout.get_session(session)
    provider = vso.phot.BatchDataProvider(session_layout)

    data = provider.batch_and_sequence_band_pair(bands)
    batched = data.group_by('batch_id').groups
    airmass = join(batched.keys, provider.batches_['batch_id', 'airmass', 'airmass_range'], 'batch_id')
    result = QTable(dict(
        batch_id=batched.keys['batch_id'],
        airmass = airmass['airmass'],
        airmass_range = airmass['airmass_range'],
        xfm=Column([batch_regression(batch, bands) for batch in batched],
                    dtype=[('Ta', ValErrDtype), ('Za', ValErrDtype), ('Tb', ValErrDtype), ('Zb', ValErrDtype)])))
    return result#[fiilter_std_all(result['xfm'], scale=.5)]
tr1 = calc_batched_classic_transform(OBJECTS[0], SESSION_TAG, bands)

vso.util.default_table_format(tr1)


In [ ]:
import matplotlib.pyplot as plt

data = [(obj, calc_batched_classic_transform(obj, SESSION_TAG, bands)) for obj in OBJECTS]

def plot_band_xfm(ax, data, T, Z):
    for obj, d in data:
        xfm = d['xfm']
        ax.errorbar(xfm[T]['val'], xfm[Z]['val'], xerr=xfm[T]['err']/2, yerr=xfm[Z]['err']/2, fmt='.', label=obj)

fig = plt.figure(figsize=(10.24, 6.4))
gs = fig.add_gridspec(1, 2)

ax = plt.subplot(gs[0,0])
plot_band_xfm(ax, data, 'Ta', 'Za')
ax.legend()
ax.set_title(bands[0])

ax = plt.subplot(gs[0,1])
plot_band_xfm(ax, data, 'Tb', 'Zb')
ax.legend()
ax.set_title(bands[1])

plt.tight_layout()
plt.show

In [ ]:
def plot_airmass_xfm(ax, data, X):
    for obj, d in data:
        xfm = d['xfm']
        ax.errorbar(d['airmass'], xfm[X]['val'], xerr=d['airmass_range']/2, yerr=xfm[X]['err']/2, fmt='.', label=obj)

fig = plt.figure(figsize=(10.24, 10.24))
gs = fig.add_gridspec(2, 2)

ax = plt.subplot(gs[0,0])
plot_airmass_xfm(ax, data, 'Ta')
ax.legend()
ax.set_title(rf'$\ T_{bands[0]} $')

ax = plt.subplot(gs[0,1])
plot_airmass_xfm(ax, data, 'Za')
ax.legend()
ax.set_title(rf'$\ Z_{bands[0]} $')

ax = plt.subplot(gs[1,0])
plot_airmass_xfm(ax, data, 'Tb')
ax.legend()
ax.set_title(rf'$\ T_{bands[1]} $')

ax = plt.subplot(gs[1,1])
plot_airmass_xfm(ax, data, 'Zb')
ax.legend()
ax.set_title(rf'$\ Z_{bands[1]} $')

plt.tight_layout()
plt.show

In [ ]:
ClassicTransform = namedtuple('ClassicTransform', ['T', 'k2', 'Z', 'k1'])
ClassicTransformDtype=[('T', ValErrDtype), ('k2', ValErrDtype), ('Z', ValErrDtype), ('k1', ValErrDtype)]

all_data = vstack([d['airmass', 'xfm'] for _, d in data])
Ta = sst.linregress(all_data['airmass'], all_data['xfm']['Ta']['val'])
Za = sst.linregress(all_data['airmass'], all_data['xfm']['Za']['val'])
Tb = sst.linregress(all_data['airmass'], all_data['xfm']['Tb']['val'])
Zb = sst.linregress(all_data['airmass'], all_data['xfm']['Zb']['val'])
tr_path = vso.util.WorkLayout(WORK_ROOT).root_dir / 'session' / SESSION_TAG / f"Tr_{bands[0]}{bands[1]}.ecsv"
session_tr = QTable({
    bands[0]: Column([ClassicTransform(ValErr(Ta.intercept, Ta.intercept_stderr), ValErr(Ta.slope, Ta.stderr),
                               ValErr(Za.intercept, Za.intercept_stderr), ValErr(Za.slope, Za.stderr))], dtype=ClassicTransformDtype),
    bands[1]: Column([ClassicTransform(ValErr(Tb.intercept, Tb.intercept_stderr), ValErr(Tb.slope, Tb.stderr),
                               ValErr(Zb.intercept, Zb.intercept_stderr), ValErr(Zb.slope, Zb.stderr))], dtype=ClassicTransformDtype)})
session_tr.write(tr_path, format='ascii.ecsv', overwrite=True)
session_tr